# Landsat8からLSTを取得するJupyter Notebook

https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#bands


## ステップ１：GoogleEarthEngineの登録

1.1. Google Earth Engine プラットフォーム([https://earthengine.google.com/)](https://earthengine.google.com/)に移動します。

1.2. Google アカウントを使用してサインインするか、アカウントをお持ちでない場合は作成します。

1.3.プロジェクトのIDをコピーする。Pythonでプログラム作成時に用います



## ステップ２：ライブラリのインストール、GEEの認証
earthengine-apiのライブラリが必要です。
また、地図上で可視化する場合、geemapも入れておくとよいです

In [86]:
from datetime import datetime
import ee
import geemap

GGE_PROJECT = 'master-research-465403'  # Google Earth EngineプロジェクトID
# Google Earth Engine APIの初期化
try:
       ee.Initialize(project=GGE_PROJECT)
except Exception as e:
       ee.Authenticate()
       ee.Initialize(project=GGE_PROJECT)

## ステップ３：ROIと期間、雲量の閾値の設定

ROIとは**Region of Interest**（＝注目領域、解析対象領域）の略称です。
本記事では、ベトナムのハノイのLSTを取得します。

行政区画の情報元:
https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level1#table-schema

In [ ]:

# GEEの行政境界データセット（GAUL）
# 参考：　https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level1#table-schema
gaul_level1 = ee.FeatureCollection("FAO/GAUL/2015/level1")
#ベトナムだけにフィルタ
vietnam_level1 = gaul_level1.filter(ee.Filter.eq('ADM0_NAME', 'Viet Nam'))
# ADM1_NAME（州名）一覧を取得
district_names = vietnam_level1.aggregate_array('ADM1_NAME').getInfo()

# 表示（重複排除＆ソート）
for name in sorted(set(district_names)):
    print(name)

ROI = ee.FeatureCollection("FAO/GAUL/2015/level1") \
         .filter(ee.Filter.eq('ADM0_NAME', 'Viet Nam')) \
         .filter(ee.Filter.eq('ADM1_NAME', 'Ha Noi City'))

# 取得するデータの期間
START_DATE = '2025-01-01'  # 開始日 例: '2025-01-01'
END_DATE = '2025-01-31'  # 終了日 例: '2025-01-31'
# 雲量の閾値（%） これ以下の画像を対象とする
CLOUD_COVER = 20 


An Giang
Ba Ria-Vung Tau
Bac Giang
Bac Kan
Bac Lieu
Bac Ninh
Ben Tre
Binh Dinh
Binh Duong
Binh Phuoc
Binh Thuan
Ca Mau
Can Tho city
Cao Bang
Da Nang City
Dak Lak
Dak Nong
Dien Bien
Dong Nai
Dong Thap
Gia Lai
Ha Giang
Ha Nam
Ha Noi City
Ha Tay
Ha Tinh
Hai Duong
Hai Phong City
Hau Giang
Ho Chi Minh City
Hoa Binh
Hung Yen
Khanh Hoa
Kien Giang
Kon Tum
Lai Chau
Lam Dong
Lang Son
Lao Cai
Long An
Nam Dinh
Nghe An
Ninh Binh
Ninh Thuan
Phu Tho
Phu Yen
Quang Binh
Quang Nam
Quang Ngai
Quang Ninh
Quang Tri
Soc Trang
Son La
Tay Ninh
Thai Binh
Thai Nguyen
Thanh Hoa
Thua Thien - Hue
Tien Giang
Tra Vinh
Tuyen Quang
Vinh Long
Vinh Phuc
Yen Bai


## ステップ4　データの取得と雲マスクの適用

### 4.1 データセットの取得

In [88]:
# Landsat 8データセット
dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate(START_DATE, END_DATE) \
    .filterBounds(ROI) \
    .filter(ee.Filter.lt('CLOUD_COVER', CLOUD_COVER))  # 雲量の閾値でフィルタリング

# LSTバンドを計算する関数
# 参考： https://www.usgs.gov/landsat-missions/landsat-collection-2-level-2-science-products
def compute_lst_and_reflectance(img):
    thermal_bands  = img.select(['ST_B10']).multiply(0.00341802).add(149.0)
    optical_bands = img.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']).multiply(0.0000275).add(-0.2)
    return thermal_bands.addBands(optical_bands).set('system:time_start', img.get('system:time_start'))

### 4.2 画像のマスク処理
衛星画像の品質評価バンド（QA_PIXEL）を使い、特定のビットが立っているピクセル（雲・影・雪）を除去

　参考：https://www.usgs.gov/media/files/landsat-8-9-collection-2-level-2-science-product-guide 

In [ ]:
def mask_clouds(img):
    qa = img.select('QA_PIXEL')
    cloud = qa.bitwiseAnd(1 << 3).eq(0)
    shadow = qa.bitwiseAnd(1 << 4).eq(0)
    snow = qa.bitwiseAnd(1 << 5).eq(0)
    mask = cloud.And(shadow).And(snow)
    return img.updateMask(mask)

元画像ピクセル数: {'QA_PIXEL': 1032180, 'QA_RADSAT': 1032180, 'SR_B1': 1032180, 'SR_B2': 1032180, 'SR_B3': 1032180, 'SR_B4': 1032180, 'SR_B5': 1032180, 'SR_B6': 1032180, 'SR_B7': 1032180, 'SR_QA_AEROSOL': 1032180, 'ST_ATRAN': 1032180, 'ST_B10': 1032180, 'ST_CDIST': 1032180, 'ST_DRAD': 1032180, 'ST_EMIS': 1032180, 'ST_EMSD': 1032180, 'ST_QA': 1032180, 'ST_TRAD': 1032180, 'ST_URAD': 1032180}
マスク後ピクセル数: {'QA_PIXEL': 1032075, 'QA_RADSAT': 1032075, 'SR_B1': 1032075, 'SR_B2': 1032075, 'SR_B3': 1032075, 'SR_B4': 1032075, 'SR_B5': 1032075, 'SR_B6': 1032075, 'SR_B7': 1032075, 'SR_QA_AEROSOL': 1032075, 'ST_ATRAN': 1032075, 'ST_B10': 1032075, 'ST_CDIST': 1032075, 'ST_DRAD': 1032075, 'ST_EMIS': 1032075, 'ST_EMSD': 1032075, 'ST_QA': 1032075, 'ST_TRAD': 1032075, 'ST_URAD': 1032075}


In [90]:
# 処理を適用
processed_dataset = dataset.map(mask_clouds).map(compute_lst_and_reflectance)


### 4.3 可視化


In [85]:
#地図上に表示
visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.15,
}
CENTER= [20.99, 105.35]  # ハノイの中心座標
Map = geemap.Map(center=CENTER, zoom=8)
Map.addLayer(ROI, {}, 'ROI')
Map.add_layer(processed_dataset, visualization, 'True Color (432)')
Map

Map(center=[20.99, 105.35], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

# ステップ5 NDVI の算出

